In [33]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore')
path="../../../ukb/data/"

'''
Helper function to find out columns containing specific text
'''
def findcols(df,text):
    return [col for col in df.columns if text in col]

def combinedata(eids,sets,df1,df2):
    df1=df1[(df1['eid'].isin(eids))]
    df2=df2[(df2['eid'].isin(eids))]
    df_out=pd.merge(df1,df2,on='eid',how='inner')
    df_out=pd.merge(df_out,sets[['eid','set']],on='eid',how='left')
    return df_out

def rebalage(df,min_age=75,max_age=83,disease="AD",diagcutoff=2,sampleratio=1,death='death',agematch=1):
    
    df['Age_Today']=df['Age_Today'].round()
    rule_age=(df['Age_Today']>=min_age)&(df['Age_Today']<=max_age)
    rule_dis=(df[disease]==1)
    rule_cont=(df[disease]==0)
    rule_death=(df[death]!=1)
    rule_diag_cut=(df['years_bef_diag'+disease]>diagcutoff)
    
    rule_nonnull=pd.notnull(df['eid'])
    mask_dis=rule_age&rule_dis&rule_diag_cut&rule_nonnull
    mask_cont=rule_age&rule_cont&rule_death&rule_nonnull
    case=df['eid'][mask_dis]
    
    if agematch==1:
        #find matches by age between disease and control - do we want age today or age attended?
        dis_age=pd.DataFrame(df[mask_dis].groupby('Age_Today').size()).reset_index()
        dis_age.columns=['Age_Today','age_recs']

        cont_age=pd.merge(df[['eid','Age_Today']][mask_cont],dis_age,on='Age_Today',how='inner')
        cont_age=pd.DataFrame(cont_age.groupby('Age_Today').\
        apply(lambda x: x['eid'].sample(x['age_recs'].max()*sampleratio,replace=False))).reset_index()
        cont_age=pd.merge(cont_age,dis_age,on='Age_Today',how='inner')
        
        cont_age['set_score']=cont_age.groupby(['Age_Today']).cumcount()/cont_age['age_recs']
        cont_age['set']=cont_age['set_score'].apply(np.floor)

        cont_age=cont_age[['eid','set']]
        control=cont_age['eid']
        
        
    else:
        
        control=df['eid'][mask_cont]
        cont_age=control.copy()
        cont_age['set']=1
    
    
    
    eids=pd.concat([case,control],axis=0)
    
    return eids,cont_age


disease_vars=['EPIL_bef','DIAB_bef','CERVASC_bef','CERVASCALL_bef','STROKE_bef','AD','EPIL','DIAB','CERVASC',\
              'CELIAC','PD','AUD','eid']

In [3]:
ukb_tp0_ohc=pd.read_pickle('%s%s' % (path,'ukb_tp0_ohc.p'))

In [22]:
diseases_bin_AUD=pd.read_pickle('%s%s' % (path,'ukb_disease_labels_AUD.p'))

In [24]:
diseases_bin_AUD.shape

(500102, 76)

In [34]:
diseases_bin_AUD['eid']=diseases_bin_AUD['n_eid']

In [35]:
AUD_model_full=pd.merge(ukb_tp0_ohc,diseases_bin_AUD[disease_vars],on='eid')

In [36]:
AUD_model_full.to_pickle('%s%s' % (path,'AUD_model_full.p'))